In [3]:
from google.colab import drive
drive.mount('/content/drive')

basicpath = '/content/drive/MyDrive/졸업프로젝트/rec/'

Mounted at /content/drive


In [4]:
import pandas as pd
import os
import numpy as np
from sklearn.svm import SVC

In [5]:
pip install surprise

     |████████████████████████████████| 11.8 MB 4.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619423 sha256=380c6d7c59332a46f9f9ffac18af437f4e1370ecf0099f513f89c2fa2ec5fda2
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [6]:
# Utilities
import math, random, warnings
from time import time
from datetime import datetime
from collections import defaultdict
from IPython.core.interactiveshell import InteractiveShell

# Mathematical calculation
import numpy as np
from scipy.sparse.linalg import svds
from sklearn import model_selection
from sklearn.metrics.pairwise import cosine_similarity

# Data handling
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# scikit-surprise recommender package
from surprise import SVD, KNNWithMeans
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.prediction_algorithms.baseline_only import BaselineOnly

# Configure for any default setting of any library
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings('ignore')
%matplotlib inline
# sns.set(style='whitegrid', palette='deep', font='sans-serif', font_scale=1.2, color_codes=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [7]:
file = 'cleaned_reviews_2018_.csv'
df = pd.read_csv(os.path.join(basicpath, file)).drop("Unnamed: 0", axis=1)

In [8]:
df.rename(columns = {'reviewerID' : 'userId', 'asin' : 'productId', 'overall' : 'rating', 'reviewTime' : 'timestamp'}, inplace = True)

In [9]:
# Get a dataframe consisting only of ratings that are duplicated
rating_combination = ['userId', 'productId']
df[df.duplicated(subset=rating_combination, keep=False)].sort_values(rating_combination).head()

,userId,productId,rating,timestamp
513830,A10DI8FE77S74Y,B017N39T58,5,"09 11, 2018"
513831,A10DI8FE77S74Y,B017N39T58,5,"09 11, 2018"
513832,A10DI8FE77S74Y,B017N39T58,5,"09 11, 2018"
513833,A10DI8FE77S74Y,B017N39T58,5,"09 11, 2018"
513834,A10DI8FE77S74Y,B017N39T58,5,"09 11, 2018"


In [10]:
# Drop the timestamp column
df.drop(labels='timestamp', axis=1, inplace=True)
df

,userId,productId,rating
0,A2ZCK9686DTWU0,106171327X,5
1,A3G5NNV6T6JA8J,106171327X,5
2,AFML7PYI3LERI,106171327X,5
3,A1G0HYMR02WM2W,106171327X,4
4,A1T8B3I8KRS3W0,106171327X,5
...,...,...,...
1087165,A34QQHRRG65E08,B01HJF9W84,5
1087166,A2ASY291VN2H6F,B01HJGOOMW,2
1087167,A3TJ82VBTRW3O6,B01HJGOOMW,1
1087168,A10GEXRHH85UDI,B01HJF704M,1


In [11]:
# Find the top 10 users based on ratings
most_rated = df.userId.value_counts().rename_axis('UserId').reset_index(name='Ratings')
most_rated.head(500)

,UserId,Ratings
0,A10SE0U42ABS9S,95
1,A2O5RXNP1U7UXC,50
2,ALDWXFCCVIGGR,47
3,A3RW697Y2EZWUL,47
4,AM2IJT2H3SG,45
...,...,...
495,A2NYSDO4SQMGE8,14
496,AUFHVRTF5074H,14
497,A1PRPHWG22CKKG,14
498,A1HS7FEPJY47CW,14


In [12]:
unique_original = (df.userId.nunique(), df.productId.nunique())

In [13]:
# Drop the users who have rated less than 6 products
df = df[df.userId.isin(most_rated[most_rated['Ratings'] >= 6].UserId)]
print('Unique USERS who have rated 6 or more products :', df.userId.nunique())
print('Unique USERS dropped      :', unique_original[0] - df.userId.nunique())
print('Unique ITEMS remaining    :', df.productId.nunique())
print('Unique ITEMS dropped      :', unique_original[1] - df.productId.nunique())
print('\nFinal length of the dataset :', len(df))

Unique USERS who have rated 6 or more products : 7934
Unique USERS dropped      : 817799
Unique ITEMS remaining    : 27646
Unique ITEMS dropped      : 96769

Final length of the dataset : 66352


In [14]:
df

,userId,productId,rating
3,A1G0HYMR02WM2W,106171327X,4
16,A2T7D9P4TM7L83,106171327X,5
17,AUGBCGUG05AH7,106171327X,5
59,A2WBQ18YJ2CYYF,B000001ON6,3
61,A288U5R1WTBGEH,B000001OM5,4
...,...,...,...
1087017,A7JA5G6OMWYSR,B01HIS365C,4
1087053,A2LJ2R6YU6VG6M,B01HITUTV0,1
1087058,A7CDO4MIXAONA,B01HIUOLZ4,5
1087100,A1QGNRP0RTECUA,B01HIZK1B2,5


In [15]:
 from sklearn.preprocessing import LabelEncoder
 labelencoder = LabelEncoder()

In [16]:
labels = []
for i in range(len(df.columns)-1): #rating은 포함 안되게
  df[df.columns[i]] = labelencoder.fit_transform(df[df.columns[i]])
  labels.append(labelencoder.classes_)

print(labels)

# for i in range(len(df.columns)-1):
#   df[df.columns[i]] = labelencoder.inverse_transform(df[df.columns[i]])
# print(df)

[array(['A0455374D4OQJ3XCOGGT', 'A100ULP2P8U71Z', 'A1012HOQZDKO80', ...,
       'AZZEPOCDCD1PN', 'AZZW780H8VJ8N', 'AZZYJH0XNZ896'], dtype=object), array(['1059950073', '106171327X', '140053271X', ..., 'B01HIWPMNW',
       'B01HIZEW1C', 'B01HIZK1B2'], dtype=object)]


In [17]:
df

,userId,productId,rating
3,942,1,4
16,3702,1,5
17,7603,1,5
59,3888,22,3
61,2511,20,4
...,...,...,...
1087017,6220,27640,4
1087053,3264,27641,1
1087058,6209,27642,5
1087100,1531,27645,5


In [18]:
users_num = np.max(df.iloc[:, 0])
items_num = np.max(df.iloc[:, 1])
shape = (users_num + 1, items_num + 1)
print(shape)

(7934, 27646)


In [19]:
X_train, X_test = model_selection.train_test_split(df, test_size=0.05, random_state=5)

In [20]:
X_train

,userId,productId,rating
1074638,1654,26991,5
613000,2204,25468,4
785216,1449,9383,1
217160,2580,12999,5
72169,7149,5434,5
...,...,...,...
943888,2301,20006,5
61096,7295,4759,5
291839,4401,15955,5
259866,332,14825,5


In [21]:
X_test

,userId,productId,rating
749817,1307,6500,5
25103,1771,2253,5
478285,1310,21764,5
93533,2437,6618,5
456038,7523,21231,5
...,...,...,...
899498,4508,17325,5
54879,1597,4397,5
53186,2209,4364,5
685897,86,256,5


In [22]:
train = X_train.values.tolist()

In [23]:
print(train)

[[1654, 26991, 5], [2204, 25468, 4], [1449, 9383, 1], [2580, 12999, 5], [7149, 5434, 5], [6965, 20377, 5], [5152, 22936, 5], [7682, 24361, 4], [7452, 7758, 5], [2132, 19488, 5], [2045, 7822, 5], [4051, 7598, 4], [384, 11290, 5], [7242, 9898, 5], [1500, 23657, 5], [3458, 16598, 5], [4633, 27455, 5], [1814, 20669, 5], [1880, 19335, 5], [2042, 26273, 5], [3973, 24090, 5], [6747, 3972, 4], [5119, 8785, 5], [1922, 12868, 4], [4086, 12624, 5], [6750, 20787, 5], [1489, 19319, 5], [5409, 9396, 3], [148, 13516, 5], [442, 11976, 5], [7169, 11777, 5], [545, 9431, 4], [3146, 14885, 5], [7190, 7284, 4], [5299, 27254, 5], [1344, 7030, 5], [1671, 17402, 5], [733, 3368, 5], [527, 17818, 5], [6540, 18325, 5], [3285, 10598, 1], [6106, 9752, 4], [4349, 21784, 5], [1891, 22347, 5], [4794, 250, 5], [7672, 3725, 2], [7055, 17358, 5], [1256, 6053, 5], [2171, 8756, 4], [5185, 13469, 5], [6721, 17829, 4], [111, 25146, 4], [7886, 11812, 5], [7489, 12486, 5], [4604, 16878, 1], [6084, 4892, 2], [7707, 8515, 3], [

In [24]:
rating_matrix = np.ndarray(shape, dtype=float)

for user, item, rating in train:
    rating_matrix[user][item] = rating
print(len(rating_matrix))
print(len(rating_matrix[0]))

7934
27646


In [25]:
pd.DataFrame(rating_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,27606,27607,27608,27609,27610,27611,27612,27613,27614,27615,27616,27617,27618,27619,27620,27621,27622,27623,27624,27625,27626,27627,27628,27629,27630,27631,27632,27633,27634,27635,27636,27637,27638,27639,27640,27641,27642,27643,27644,27645
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7929,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7930,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7931,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7932,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

In [ ]:
import numpy as np
global res

class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """

        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):

            # rating이 존재하는 index를 기준으로 training
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0:
                        self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        xi, yi = self._R.nonzero()
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2)
        return np.sqrt(cost / len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        global res
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        res = self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)


    def print_results(self):
        """
        print fit results
        """

        print("User Latent P:")
        print(self._P)
        print("Item Latent Q:")
        print(self._Q.T)
        print("P x Q:")
        print(self._P.dot(self._Q.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_P)
        print("Item Latent bias:")
        print(self._b_Q)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])


# run example
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)


    # P, Q is (7 X k), (k X 5) matrix
    factorizer = MatrixFactorization(rating_matrix, k=8, learning_rate=0.1, reg_param=0.1, epochs=200, verbose=True)
    factorizer.fit()
    factorizer.print_results()

Iteration: 10 ; cost = 0.1912
Iteration: 20 ; cost = 0.1917
Iteration: 30 ; cost = 0.1935
Iteration: 40 ; cost = 0.1943


In [ ]:
res.shape

In [ ]:
print(np.max(res[0]))
print(np.min(res[0]))

In [ ]:
pd.DataFrame(res)

In [ ]:
inv = pd.DataFrame(res)
inv.columns = labels[1]
inv.index = labels[0]

print(inv.shape)

#inv = inv.astype(int)

print(inv)


# lst = []
# cnt = 0
# for i in inv.columns:
#   for j in inv.index:
#     if inv[i][j] != 0 and inv[i][j] != "nan":
#       lst.append([i, j, inv[i][j]])
#       cnt += 1

# print(lst)



In [ ]:
inv

In [ ]:
#inv.to_csv(basicpath + 'rating8_iter10.csv', mode='w', encoding='utf-8', sep = '\t')

In [ ]:
# inv.loc['A10SE0U42ABS9S']
# inv.loc['A3RW697Y2EZWUL']
# inv.loc['ALDWXFCCVIGGR']
# inv.loc['AM2IJT2H3SG']
# inv.loc['A2ZDJWO9G0CTLL']
# inv.loc['A37XP21TPO5ZED']
# inv.loc['A2NF3AZ6GSIBEY']
# inv.loc['A22BQ2P4R61T3E']
# inv.loc['AH8610KJ5D5T7']

# ['A10SE0U42ABS9S']
# ['A3RW697Y2EZWUL']
# ['ALDWXFCCVIGGR']
# ['AM2IJT2H3SG']
# ['A2ZDJWO9G0CTLL']
# ['A37XP21TPO5ZED']
# ['A2NF3AZ6GSIBEY']
# ['A22BQ2P4R61T3E']
# ['AH8610KJ5D5T7']


In [ ]:

inv.loc['A10SE0U42ABS9S'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec1.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2O5RXNP1U7UXC'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec3.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A3RW697Y2EZWUL'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec2.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['ALDWXFCCVIGGR'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec4.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['AM2IJT2H3SG'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec5.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2ZDJWO9G0CTLL'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec6.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A37XP21TPO5ZED'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec7.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A22BQ2P4R61T3E'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec8.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2NF3AZ6GSIBEY'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec9.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A1G0HYMR02WM2W'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec10.txt', mode='w', encoding='utf-8', sep = '\t')


inv.loc['A1NSAEZV6AQP2X'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec11.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A1O09DT1PREH9Y'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec12.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['ANAGQP0IXIN9M'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec13.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A3JEVM5MHKWMYS'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec14.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2LU3XZ7NZOMVV'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec15.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A3MGFV3ZX053AF'].sort_values(ascending=False)[:10].to_csv(basicpath + 'rec16.txt', mode='w', encoding='utf-8', sep = '\t')


inv.loc['A10SE0U42ABS9S'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec21.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2O5RXNP1U7UXC'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec23.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A3RW697Y2EZWUL'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec22.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['ALDWXFCCVIGGR'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec24.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['AM2IJT2H3SG'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec25.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2ZDJWO9G0CTLL'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec26.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A37XP21TPO5ZED'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec27.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A22BQ2P4R61T3E'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec28.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2NF3AZ6GSIBEY'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec29.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A1G0HYMR02WM2W'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec30.txt', mode='w', encoding='utf-8', sep = '\t')


inv.loc['A1NSAEZV6AQP2X'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec31.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A1O09DT1PREH9Y'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec32.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['ANAGQP0IXIN9M'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec33.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A3JEVM5MHKWMYS'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec34.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A2LU3XZ7NZOMVV'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec35.txt', mode='w', encoding='utf-8', sep = '\t')
inv.loc['A3MGFV3ZX053AF'].sort_values(ascending=True)[:10].to_csv(basicpath + 'rec36.txt', mode='w', encoding='utf-8', sep = '\t')

In [ ]:
testing = X_test.values.tolist()
total = 0
fault_cnt = 0
for user, item, rating in testing:
  try:
    if inv.at[labels[0][user],labels[1][item]] > 5:
      total += pow(5 - rating, 2)
    elif inv.at[labels[0][user],labels[1][item]] < 0:
      total += pow(0 - rating, 2)
    else:
      total += pow(inv.at[labels[0][user],labels[1][item]] - rating, 2)
  except:
    fault_cnt += 1
print(len(testing))
print("fault : " + str(fault_cnt))
print("<rmse result>")
print(np.sqrt(total / (len(testing)-fault_cnt)))

In [ ]:
testing = X_test.values.tolist()
total = 0
fault_cnt = 0
for user, item, rating in testing:
  try:
    if inv.at[labels[0][user],labels[1][item]] > 5:
      total += pow(5 - rating, 2)
    elif inv.at[labels[0][user],labels[1][item]] < 0:
      total += pow(0 - rating, 2)
    else:
      total += pow(inv.at[labels[0][user],labels[1][item]] - rating, 2)
  except:
    fault_cnt += 1
print(len(testing))
print("fault : " + str(fault_cnt))
print("<rmse result>")
print(np.sqrt(total / (len(testing)-fault_cnt)))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_inv = min_max_scaler.fit_transform(inv)

scale_inv = min_max_inv * 5

total = 0
fault_cnt = 0
for user, item, rating in testing:
  try:
      total += pow(inv.at[labels[0][user],labels[1][item]] - rating, 2)
  except:
    fault_cnt += 1
print(len(testing))
print("fault : " + str(fault_cnt))
print("<rmse result>")
print(np.sqrt(total / (len(testing)-fault_cnt)))


In [ ]:

scale_inv.loc['A10SE0U42ABS9S'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec1.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2O5RXNP1U7UXC'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec3.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A3RW697Y2EZWUL'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec2.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['ALDWXFCCVIGGR'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec4.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['AM2IJT2H3SG'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec5.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2ZDJWO9G0CTLL'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec6.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A37XP21TPO5ZED'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec7.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A22BQ2P4R61T3E'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec8.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2NF3AZ6GSIBEY'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec9.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A1G0HYMR02WM2W'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec10.txt', mode='w', encoding='utf-8', sep = '\t')


scale_inv.loc['A1NSAEZV6AQP2X'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec11.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A1O09DT1PREH9Y'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec12.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['ANAGQP0IXIN9M'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec13.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A3JEVM5MHKWMYS'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec14.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2LU3XZ7NZOMVV'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec15.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A3MGFV3ZX053AF'].sort_values(ascending=False)[:10].to_csv(basicpath + 'scale_rec16.txt', mode='w', encoding='utf-8', sep = '\t')


scale_inv.loc['A10SE0U42ABS9S'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec21.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2O5RXNP1U7UXC'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec23.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A3RW697Y2EZWUL'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec22.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['ALDWXFCCVIGGR'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec24.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['AM2IJT2H3SG'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec25.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2ZDJWO9G0CTLL'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec26.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A37XP21TPO5ZED'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec27.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A22BQ2P4R61T3E'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec28.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2NF3AZ6GSIBEY'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec29.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A1G0HYMR02WM2W'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec30.txt', mode='w', encoding='utf-8', sep = '\t')


scale_inv.loc['A1NSAEZV6AQP2X'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec31.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A1O09DT1PREH9Y'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec32.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['ANAGQP0IXIN9M'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec33.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A3JEVM5MHKWMYS'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec34.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A2LU3XZ7NZOMVV'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec35.txt', mode='w', encoding='utf-8', sep = '\t')
scale_inv.loc['A3MGFV3ZX053AF'].sort_values(ascending=True)[:10].to_csv(basicpath + 'scale_rec36.txt', mode='w', encoding='utf-8', sep = '\t')